In [1]:
import torch
import random
from datasets import load_dataset
from sklearn.metrics import accuracy_score
from transformers import AutoModelForCausalLM, AutoTokenizer


In [2]:
if torch.cuda.is_available():
    device = "cuda"

print(device)

model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", trust_remote_code=True).to(device)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)


cuda


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
dataset = load_dataset("financial_phrasebank", 'sentences_allagree')
dataset = dataset['train']


In [60]:
random.seed(10)
sampled_indices = random.sample(range(len(dataset)), 1000)
sampled_data = [dataset[i] for i in sampled_indices]

sampled_sentences = [data['sentence'] for data in sampled_data]
sampled_labels = [data['label'] for data in sampled_data]


In [73]:
prompt = "Instruct: As a Financial News Article Sentiment Classification expert, analyze and classify the sentiment of the following news article as a single number, 0 being 'Negative', 1 being 'Neutral', and 2 being 'Positive'. Provide only a single number as output, without additional text. Below is a sentiment classification example:\n Example: 'Operating margin , however , slipped to 14.4 % from 15.1 % , dragged down by a poor performance in enterprise solutions .' - Output:0 \n Example: 'In January-September 2009 , the Group 's net interest income increased to EUR 112.4 mn from EUR 74.3 mn in January-September 2008 .' - Output:2 \n Sentiment Classification news article: "


In [74]:
predictions = []

for sentence in sampled_sentences:
    inputs = tokenizer([prompt + "'" + sentence + "' \n Output:"], return_tensors="pt", return_attention_mask=False).to(device)
    outputs = model.generate(**inputs, max_new_tokens=1, pad_token_id=model.config.eos_token_id)
    text = tokenizer.batch_decode(outputs)
    predictions.append(int(text[-1][-1]))



In [75]:
accuracy = accuracy_score(sampled_labels, predictions)
print(accuracy)

0.723
